In [68]:
from glob import glob
import pandas as pd
paths = glob('./data/*/*/*')
filedata = pd.DataFrame([path.split('/')[2:] for path in paths], columns=['subject', 'pose', 'filename'])
filedata['path'] = paths
filedata = filedata.sort_values(['subject', 'pose']).reset_index(drop=True)
filedata
data = []
for index, value in filedata.iterrows():
    df = pd.read_pickle(value['path'])
    df = df.rename_axis('time').reset_index()
    df['id'] = index
    cols = df.columns.to_list()
    df = df[cols[-1:] + cols[:-1]]
    data.append(df)
data = pd.concat(data)
data = data.reset_index(drop=True)
data

Category  id time force                             x_coord      ...  \
Position              A    B     C      D    E    F       A   B  ...   
0          0    0   0.0  0.0   0.0   22.0  0.0  0.0     NaN NaN  ...   
1          0    1   0.0  0.0   0.0  277.0  0.0  0.0     NaN NaN  ...   
2          0    2   0.0  0.0   0.0  488.0  0.0  0.0     NaN NaN  ...   
3          0    3   0.0  0.0   0.0  501.0  0.0  0.0     NaN NaN  ...   
4          0    4   0.0  0.0   0.0  540.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...   ...    ...  ...  ...     ...  ..  ...   
52144     72  635   0.0  0.0   0.0  983.0  0.0  0.0     NaN NaN  ...   
52145     72  636   0.0  0.0   0.0  962.0  0.0  0.0     NaN NaN  ...   
52146     72  637   0.0  0.0   0.0  910.0  0.0  0.0     NaN NaN  ...   
52147     72  638   0.0  0.0   0.0  851.0  0.0  0.0     NaN NaN  ...   
52148     72  639   0.0  0.0  10.0  839.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                  FSR_for_coord                 \
Position             C      D    E    F             A    B         C   
0                  0.0    0.0  0.0  0.0           0.0  0.0  0.000000   
1                  0.0   60.0  0.0  0.0           0.0  0.0  0.000000   
2                  0.0   73.0  0.0  0.0           0.0  0.0  0.000000   
3                  0.0   84.0  0.0  0.0           0.0  0.0  0.000000   
4                  0.0  100.0  0.0  0.0           0.0  0.0  0.000000   
...                ...    ...  ...  ...           ...  ...       ...   
52144             37.0  180.0  0.0  0.0           0.0  0.0  0.587302   
52145             38.0  172.0  0.0  0.0           0.0  0.0  0.603175   
52146             38.0  165.0  0.0  0.0           0.0  0.0  0.603175   
52147             39.0  160.0  0.0  0.0           0.0  0.0  0.619048   
52148             39.0  156.0  0.0  0.0           0.0  0.0  0.619048   

Category                      
Position         D    E    F  
0         0.000000  0.0  0.0  
1         0.952381  0.0  0.0  
2         1.158730  0.0  0.0  
3         1.333333  0.0  0.0  
4         1.587302  0.0  0.0  
...            ...  ...  ...  
52144     2.857143  0.0  0.0  
52145     2.730159  0.0  0.0  
52146     2.619048  0.0  0.0  
52147     2.539683  0.0  0.0  
52148     2.476190  0.0  0.0  

[52149 rows x 32 columns]

In [222]:
import torch

class FSRDataset(torch.utils.data.Dataset):
    def __init__(self, X_df, y_df, index):
        assert(len(X_df) == len(y_df))
        self.X_df = X_df
        self.y_df = y_df
        self.index = index

    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, idx):
        import numpy as np
        X = self.X_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        y = self.y_df.loc[self.index[idx]].to_numpy().astype(np.float32)
        return X, y

In [118]:
def get_index_splited_by_time(test_size=None):
    from sklearn.model_selection import TimeSeriesSplit, train_test_split
    fold = TimeSeriesSplit()
    id_set = set(data['id'])
    train_indexes = []
    test_indexes = []
    for id in id_set:
        df = data[data['id']==id]
        train_df, test_df = train_test_split(df, test_size=test_size)
        bias = data.index[data['id']==id][0]
        train_index = [(X + bias, y + bias) for X, y in fold.split(train_df)]
        test_index = [(X + bias, y + bias) for X, y in fold.split(test_df)]
        train_indexes.extend(train_index)
        test_indexes.extend(test_index)
    return train_indexes, test_indexes
train, test = get_index_splited_by_time()

In [146]:
train_test_split(df, test_size=0.2, shuffle=False)[0]

Category  id time force                           x_coord      ...  \
Position              A    B    C     D    E    F       A   B  ...   
51509     72    0   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   
51510     72    1   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   
51511     72    2   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   
51512     72    3   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   
51513     72    4   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   
...       ..  ...   ...  ...  ...   ...  ...  ...     ...  ..  ...   
52016     72  507   0.0  0.0  0.0  60.0  0.0  0.0     NaN NaN  ...   
52017     72  508   0.0  0.0  0.0  50.0  0.0  0.0     NaN NaN  ...   
52018     72  509   0.0  0.0  0.0  38.0  0.0  0.0     NaN NaN  ...   
52019     72  510   0.0  0.0  0.0  35.0  0.0  0.0     NaN NaN  ...   
52020     72  511   0.0  0.0  0.0   0.0  0.0  0.0     NaN NaN  ...   

Category FSR_for_force                 FSR_for_coord                           \
Position             C     D    E    F             A    B         C         D   
51509             38.0  26.0  0.0  0.0           0.0  0.0  0.603175  0.412698   
51510             40.0   0.0  0.0  0.0           0.0  0.0  0.634921  0.000000   
51511             41.0   0.0  0.0  0.0           0.0  0.0  0.650794  0.000000   
51512             42.0   0.0  0.0  0.0           0.0  0.0  0.666667  0.000000   
51513             42.0   0.0  0.0  0.0           0.0  0.0  0.666667  0.000000   
...                ...   ...  ...  ...           ...  ...       ...       ...   
52016              0.0  66.0  0.0  0.0           0.0  0.0  0.000000  1.047619   
52017              0.0  64.0  0.0  0.0           0.0  0.0  0.000000  1.015873   
52018              0.0  61.0  0.0  0.0           0.0  0.0  0.000000  0.968254   
52019              0.0  57.0  0.0  0.0           0.0  0.0  0.000000  0.904762   
52020              0.0  54.0  0.0  0.0           0.0  0.0  0.000000  0.857143   

Category            
Position    E    F  
51509     0.0  0.0  
51510     0.0  0.0  
51511     0.0  0.0  
51512     0.0  0.0  
51513     0.0  0.0  
...       ...  ...  
52016     0.0  0.0  
52017     0.0  0.0  
52018     0.0  0.0  
52019     0.0  0.0  
52020     0.0  0.0  

[512 rows x 32 columns]

In [155]:
from sklearn.model_selection import train_test_split
id_set = set(data['id'])
train_indexes = []
for id in id_set:
    df = data[data['id']==id]
    train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
    bias = data.index[data['id']==id][0]
    train_index = [(X + bias, y + bias) for X, y in fold.split(train_df)]
    test_index = [(X + bias, y + bias) for X, y in fold.split(test_df)]
    train_indexes.extend(train_index)
    test_indexes.extend(test_index)


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75]),
 array([ 76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
         89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
        102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150]))

In [174]:
def get_index_splited_by_time(test_size=None):
    train_indexes = []
    test_indexes = []
    for _, group in data.groupby('id'):
        train_index, test_index = train_test_split(group.index, test_size=0.2, shuffle=False)
        train_indexes.append(train_index)
        test_indexes.append(test_index)
    return train_indexes, test_indexes

In [224]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layer, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.output_size = output_size
        self.encoder = torch.nn.LSTM(input_size, hidden_size, num_layer)
        self.decoder = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x, _ = self.encoder(x)
        x = self.decoder(x)
        return x
    
train_dataset = FSRDataset(data['FSR_for_force'], data['force'], train_indexes)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
model = LSTM(**{
            'input_size':6, 
            'hidden_size':128, 
            'num_layer':4, 
            'output_size':6,
        })
a = iter(train_loader)
next(a)
next(a)
# for X, y in train_loader:
#     pred = model(X)

[tensor([[223., 270., 179.,   0.,  44.,   0.],
         [227., 289., 187.,   0.,  46.,   0.],
         [230., 309., 191.,   0.,  49.,   0.],
         ...,
         [127., 123., 317.,   0.,  49.,  67.],
         [131., 126., 332.,   0.,  50.,  67.],
         [133., 127., 342.,   0.,  50.,  64.]]),
 tensor([[ 232.,  941.,  758.,    0.,   35.,    0.],
         [ 251.,  974.,  792.,    0.,   37.,    3.],
         [ 246.,  984.,  824.,    0.,   34.,    0.],
         ...,
         [  93.,  685., 1232.,    0.,   39.,  136.],
         [  87.,  665., 1284.,    0.,   26.,  129.],
         [  93.,  682., 1338.,    0.,   32.,  111.]])]

In [193]:
train_dataset[0][0].shape

(451, 6)

In [195]:
train_indexes[1]

Index([ 564,  565,  566,  567,  568,  569,  570,  571,  572,  573,
       ...
       1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053],
      dtype='int64', length=490)

In [186]:
data.iloc[train_indexes[1]]


Category id time  force                                     x_coord            \
Position              A       B       C    D     E      F         A         B   
564       1    0  232.0   941.0   758.0  0.0  35.0    0.0  8.396552  7.513326   
565       1    1  251.0   974.0   792.0  0.0  37.0    3.0  8.372510  7.508727   
566       1    2  246.0   984.0   824.0  0.0  34.0    0.0  8.339431  7.505607   
567       1    3  249.0  1012.0   802.0  0.0  38.0    0.0  8.381526  7.505952   
568       1    4  257.0  1036.0   813.0  0.0  44.0    0.0  8.383268  7.516457   
...      ..  ...    ...     ...     ...  ...   ...    ...       ...       ...   
1049      1  485   61.0   616.0  1013.0  0.0   8.0  141.0  8.368852  7.315747   
1050      1  486   73.0   653.0  1126.0  0.0  18.0  145.0  8.219178  7.322358   
1051      1  487   93.0   685.0  1232.0  0.0  39.0  136.0  8.145161  7.296350   
1052      1  488   87.0   665.0  1284.0  0.0  26.0  129.0  8.189655  7.253383   
1053      1  489   93.0   682.0  1338.0  0.0  32.0  111.0  8.193548  7.251466   

Category  ... FSR_for_force                   FSR_for_coord            \
Position  ...             C     D     E     F             A         B   
564       ...         179.0   0.0  44.0   0.0      3.539683  4.285714   
565       ...         187.0   0.0  46.0   0.0      3.603175  4.587302   
566       ...         191.0   0.0  49.0   0.0      3.650794  4.904762   
567       ...         195.0   0.0  52.0   0.0      3.682540  5.238095   
568       ...         200.0   0.0  52.0   0.0      3.714286  5.571429   
...       ...           ...   ...   ...   ...           ...       ...   
1049      ...         268.0  51.0  46.0  61.0      1.825397  1.920635   
1050      ...         296.0   0.0  48.0  64.0      1.920635  1.936508   
1051      ...         317.0   0.0  49.0  67.0      2.015873  1.952381   
1052      ...         332.0   0.0  50.0  67.0      2.079365  2.000000   
1053      ...         342.0   0.0  50.0  64.0      2.111111  2.015873   

Category                                          
Position         C         D         E         F  
564       2.841270  0.000000  0.698413  0.000000  
565       2.968254  0.000000  0.730159  0.000000  
566       3.031746  0.000000  0.777778  0.000000  
567       3.095238  0.000000  0.825397  0.000000  
568       3.174603  0.000000  0.825397  0.000000  
...            ...       ...       ...       ...  
1049      4.253968  0.809524  0.730159  0.968254  
1050      4.698413  0.000000  0.761905  1.015873  
1051      5.031746  0.000000  0.777778  1.063492  
1052      5.269841  0.000000  0.793651  1.063492  
1053      5.428571  0.000000  0.793651  1.015873  

[490 rows x 32 columns]

In [229]:
import torch
from ray.air import session
from ray.air.config import ScalingConfig
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ray.train.torch
import numpy as np
import random
import fsr_model as net
import torch.utils.data

def train_loop_per_worker(config):
    ray.train.torch.enable_reproducibility()
    model_name = config['model_name']
    model_args = config['model_args']
    num_epochs = config['num_epochs']
    criterion_name = config['criterion_name']
    optimizer_name = config['optimizer_name']
    lr = config['lr']

    model = getattr(net, model_name)(**model_args)
    model = ray.train.torch.prepare_model(model)
    criterion = getattr(torch.nn, criterion_name)()
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    train_index, test_index = get_index_splited_by_time()
    train_dataset = FSRDataset(data['FSR_for_force'], data['force'], train_index)
    test_dataset = FSRDataset(data['FSR_for_force'], data['force'], test_index)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None)
    train_loader = ray.train.torch.prepare_data_loader(train_loader)
    test_loader = ray.train.torch.prepare_data_loader(test_loader)

    best_rmse = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        criterion.train()
        for X, y in train_loader:
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        criterion.eval()
        with torch.no_grad():
            mae = []
            mse = []
            num = []
            for X, y in test_loader:
                pred = model(X)
                mae.append(mean_absolute_error(y, pred.cpu().detach()))
                mse.append(mean_squared_error(y, pred.cpu().detach()))
                num.append(len(y))
            mae = np.array(mae)
            mse = np.array(mse)
            num = np.array(num)
            mae = (mae * num).sum() / sum(num)
            mse = (mse * num).sum() / sum(num)
            
            rmse = mse ** 0.5
            if rmse < best_rmse:
                best_rmse = rmse

            session.report({'MAE': mae, 'RMSE': rmse})
                
    return best_rmse

trainer = ray.train.torch.TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        'batch_size':128,
        'lr':0.0001,
        'model_name':'LSTM',
        'model_args':{
            'input_size':6, 
            'hidden_size':128, 
            'num_layer':4, 
            'output_size':6,
        },
        'num_epochs': 32,
        'criterion_name': 'MSELoss',
        'optimizer_name': 'Adam',
    },
    scaling_config=ScalingConfig(
        num_workers=3,
        use_gpu=False,
        trainer_resources={'CPU':0}
    ),
)

In [230]:
trainer.fit()

(TorchTrainer pid=81614) 2023-06-28 10:21:17,394	INFO backend_executor.py:137 -- Starting distributed worker processes: ['81662 (172.26.215.93)', '81663 (172.26.215.93)', '81664 (172.26.215.93)']
(RayTrainWorker pid=81337) /tmp/ipykernel_68778/3441519681.py:56: RuntimeWarning: invalid value encountered in scalar divide [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayTrainWorker pid=81662) 2023-06-28 10:21:19,026	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=3]
(RayTrainWorker pid=81662) 2023-06-28 10:21:20,224	INFO train_loop_utils.py:286 -- Moving model to device: cpu
(RayTrainWorker pid=81662) 2023-06-28 10:21:20,225	INFO train_loop_utils.py:346 -- Wrapping provided model in DistributedDataParallel.


Trial name,MAE,RMSE,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
TorchTrainer_117a9_00000,227.397,525.853,2023-06-28_10-22-56,True,0,DESKTOP-0P789CI,32,172.26.215.93,81614,100.682,2.87387,100.682,1687915376,32,117a9_00000


2023-06-28 10:22:58,115	INFO tune.py:1111 -- Total run time: 106.15 seconds (106.12 seconds for the tuning loop).


Result(
  metrics={'MAE': 227.39678746011765, 'RMSE': 525.8533704863545, 'done': True, 'trial_id': '117a9_00000', 'experiment_tag': '0'},
  path='/home/seokj/ray_results/TorchTrainer_2023-06-28_10-21-11/TorchTrainer_117a9_00000_0_2023-06-28_10-21-11',
  checkpoint=None
)